## Instagram Scraper

In [1]:
# import dependencies 
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [2]:
#Read Yelp Museums, reset index(to keep original), show the dataFrame
yelp_museum_df = pd.read_csv('yelp_museums.csv')
yelp_museum_df.reset_index(inplace=True, drop=True)
yelp_museum_df[['museum names', 'museum names no space', 'museum ratings', 'number of museum reviews']]

,museum names,museum names no space,museum ratings,number of museum reviews
0,Smithsonian National Museum of Natural History,SmithsonianNationalMuseumofNaturalHistory,4.5,1002
1,Newseum,Newseum,4.5,920
2,Smithsonian National Air and Space Museum,SmithsonianNationalAirandSpaceMuseum,4.0,1066
3,National Gallery of Art,NationalGalleryofArt,4.5,653
4,National Museum of American History,NationalMuseumofAmericanHistory,4.0,629
5,United States Holocaust Memorial Museum,UnitedStatesHolocaustMemorialMuseum,4.5,753
6,National Museum of African American History an...,NationalMuseumofAfricanAmericanHistoryandCulture,4.5,763
7,United States Botanic Garden,UnitedStatesBotanicGarden,4.5,456
8,National Portrait Gallery,NationalPortraitGallery,4.5,554
9,Smithsonian American Art Museum Renwick Gallery,SmithsonianAmericanArtMuseumRenwickGallery,4.5,409


In [3]:
# get each individual instagram url
hashtags = yelp_museum_df['museum names no space']
browser = webdriver.Chrome('PATH\TO\DRIVER')

# create empty lists to append later 
links = []
locations = []

# loop through each of the hashtags 
for hashtag in hashtags:
    browser.get('https://www.instagram.com/explore/tags/' + hashtag)
    
    # parse html source
    source = browser.page_source
    data = bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    
    # loop through the links and generate the unique url for each ig post
    for link in script.findAll('a'):
        if re.match("/p", link.get('href')):
            links.append('https://www.instagram.com' + link.get('href'))
            locations.append(hashtag)
            
    Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [4]:
# parse out the number of comments and likes

# create empty lists to append to later
num_comments = []
num_likes = []

for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data = bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data = json.loads(raw)
        posts = json_data['entry_data']['PostPage'][0]['graphql']
        
        # parse out the number of comments for each post
        comments = posts['shortcode_media']['edge_media_to_parent_comment']['count']
        num_comments.append(comments)
        
        # parse out the number of likes for each post
        likes = posts['shortcode_media']['edge_media_preview_like']['count']
        num_likes.append(likes)
       
    except:
        np.nan

In [5]:
# do not want to grab the first ig link since it is the username...
# when i do this it cuts off the last one too for some reasonnnnnn
print(len(links))
print(len(num_likes))
print(len(num_comments))
print(len(locations))

377
333
333
377


In [12]:
insta_df = pd.DataFrame({'location': locations[0:333], 
                         'links': links[0:333],
                         'number of likes': num_likes, 
                         'number of comments': num_comments
                        })

insta_df

,location,links,number of likes,number of comments
0,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1YrcFLgIQv/,31,1
1,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1R3wjLnX5z/,24,4
2,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1UftUGnXA8/,44,2
3,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1Z5NUchXuM/,192,1
4,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/By_GQZMhEbS/,14,2
5,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/By-Pz2OhEvK/,87,4
6,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/By6qlD5hP1_/,13,2
7,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B0mW9QIJ21A/,124,14
8,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/Bl-z0eiBXAn/,25,0
9,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B17uVecnMtK/,21,0


In [13]:
# export df as a csv file
insta_df.to_csv(r'ig_data.csv')

In [14]:
# get the sum of the numnber of likes and number of comments grouped by location
grouped_insta_df = insta_df.groupby(['location'])['number of likes', 'number of comments'].sum()
grouped_insta_df

,number of likes,number of comments
location,,
HirshhornMuseumSculptureGarden,60,3
InternationalSpyMuseum,21975,269
MuseumoftheBible,3187,70
NationalGalleryofArt,2294,120
NationalGeographicMuseum,376,25
NationalMuseumofAfricanAmericanHistoryandCulture,1485,115
NationalMuseumofAmericanHistory,1813,65
NationalPortraitGallery,3691,569
Newseum,5233,142


In [15]:
# export grouped df as a csv
grouped_insta_df.to_csv(r'ig_data_summary.csv')